In [ ]:
cd ..

In [1]:
!pip3 install .
!pip3 install OpenHowNet
!pip3 install tensorflow_text
import tensorflow_text


ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [11]:
import textattack
import transformers

def ChineseMorphemeAttack(model_wrapper):
    import textattack
    import transformers
    import OpenHowNet
    import string

    class ChineseWordSwapHowNet(textattack.transformations.WordSwap):
        """Transforms an input by replacing its words with synonyms provided by
        WordNet."""

        def __init__(self):
            self.hownet_dict = OpenHowNet.HowNetDict(use_sim=True)
            self.topk = 10

        def _get_replacement_words(self, word):
            """Returns a list containing all possible words with 1 character
            replaced by a homoglyph."""
            
            if self.hownet_dict.get(word):
                results = self.hownet_dict.get_nearest_words_via_sememes(word, self.topk)
                synonyms = [w["word"] for r in results for w in r["synset"] if w["word"] != word]
                return synonyms
            else:
                return []


    wordnet_transformation = textattack.transformations.WordSwapWordNet(language="cmn")
    hownet_transformation = ChineseWordSwapHowNet()
    lm_model = transformers.BertForMaskedLM.from_pretrained("hfl/chinese-bert-wwm-ext")
    tokenizer = transformers.BertTokenizerFast.from_pretrained("hfl/chinese-bert-wwm-ext")
    masked_lm_transformation = textattack.transformations.WordSwapMaskedLM(
            masked_language_model=lm_model, 
            tokenizer=tokenizer,
            max_candidates=20,
            max_length=128, 
#             window_size=20,
            min_confidence=0.0,
        )   
    
    transformations = textattack.transformations.CompositeTransformation([
        wordnet_transformation, 
        hownet_transformation, 
        masked_lm_transformation
    ])

    stopwords = set(
        ["、", "。", "〈", "〉", "《", "》", "一", "一个", "一些", "一何", "一切", "一则", "一方面", "一旦", "一来", "一样", "一种", "一般", "一转眼", "七", "万一", "三", "上", "上下", "下", "不", "不仅", "不但", "不光", "不单", "不只", "不外乎", "不如", "不妨", "不尽", "不尽然", "不得", "不怕", "不惟", "不成", "不拘", "不料", "不是", "不比", "不然", "不特", "不独", "不管", "不至于", "不若", "不论", "不过", "不问", "与", "与其", "与其说", "与否", "与此同时", "且", "且不说", "且说", "两者", "个", "个别", "中", "临", "为", "为了", "为什么", "为何", "为止", "为此", "为着", "乃", "乃至", "乃至于", "么", "之", "之一", "之所以", "之类", "乌乎", "乎", "乘", "九", "也", "也好", "也罢", "了", "二", "二来", "于", "于是", "于是乎", "云云", "云尔", "五", "些", "亦", "人", "人们", "人家", "什", "什么", "什么样", "今", "介于", "仍", "仍旧", "从", "从此", "从而", "他", "他人", "他们", "他们们", "以", "以上", "以为", "以便", "以免", "以及", "以故", "以期", "以来", "以至", "以至于", "以致", "们", "任", "任何", "任凭", "会", "似的", "但", "但凡", "但是", "何", "何以", "何况", "何处", "何时", "余外", "作为", "你", "你们", "使", "使得", "例如", "依", "依据", "依照", "便于", "俺", "俺们", "倘", "倘使", "倘或", "倘然", "倘若", "借", "借傥然", "假使", "假如", "假若", "做", "像", "儿", "先不先", "光", "光是", "全体", "全部", "八", "六", "兮", "共", "关于", "关于具体地说", "其", "其一", "其中", "其二", "其他", "其余", "其它", "其次", "具体地说", "具体说来", "兼之", "内", "再", "再其次", "再则", "再有", "再者", "再者说", "再说", "冒", "冲", "况且", "几", "几时", "凡", "凡是", "凭", "凭借", "出于", "出来", "分", "分别", "则", "则甚", "别", "别人", "别处", "别是", "别的", "别管", "别说", "到", "前后", "前此", "前者", "加之", "加以", "区", "即", "即令", "即使", "即便", "即如", "即或", "即若", "却", "去", "又", "又及", "及", "及其", "及至", "反之", "反而", "反过来", "反过来说", "受到", "另", "另一方面", "另外", "另悉", "只", "只当", "只怕", "只是", "只有", "只消", "只要", "只限", "叫", "叮咚", "可", "可以", "可是", "可见", "各", "各个", "各位", "各种", "各自", "同", "同时", "后", "后者", "向", "向使", "向着", "吓", "吗", "否则", "吧", "吧哒", "含", "吱", "呀", "呃", "呕", "呗", "呜", "呜呼", "呢", "呵", "呵呵", "呸", "呼哧", "咋", "和", "咚", "咦", "咧", "咱", "咱们", "咳", "哇", "哈", "哈哈", "哉", "哎", "哎呀", "哎哟", "哗", "哟", "哦", "哩", "哪", "哪个", "哪些", "哪儿", "哪天", "哪年", "哪怕", "哪样", "哪边", "哪里", "哼", "哼唷", "唉", "唯有", "啊", "啐", "啥", "啦", "啪达", "啷当", "喂", "喏", "喔唷", "喽", "嗡", "嗡嗡", "嗬", "嗯", "嗳", "嘎", "嘎登", "嘘", "嘛", "嘻", "嘿", "嘿嘿", "四", "因", "因为", "因了", "因此", "因着", "因而", "固然", "在", "在下", "在于", "地", "基于", "处在", "多", "多么", "多少", "大", "大家", "她", "她们", "好", "如", "如上", "如上所述", "如下", "如何", "如其", "如同", "如是", "如果", "如此", "如若", "始而", "孰料", "孰知", "宁", "宁可", "宁愿", "宁肯", "它", "它们", "对", "对于", "对待", "对方", "对比", "将", "小", "尔", "尔后", "尔尔", "尚且", "就", "就是", "就是了", "就是说", "就算", "就要", "尽", "尽管", "尽管如此", "岂但", "己", "已", "已矣", "巴", "巴巴", "年", "并", "并且", "庶乎", "庶几", "开外", "开始", "归", "归齐", "当", "当地", "当然", "当着", "彼", "彼时", "彼此", "往", "待", "很", "得", "得了", "怎", "怎么", "怎么办", "怎么样", "怎奈", "怎样", "总之", "总的来看", "总的来说", "总的说来", "总而言之", "恰恰相反", "您", "惟其", "慢说", "我", "我们", "或", "或则", "或是", "或曰", "或者", "截至", "所", "所以", "所在", "所幸", "所有", "才", "才能", "打", "打从", "把", "抑或", "拿", "按", "按照", "换句话说", "换言之", "据", "据此", "接着", "故", "故此", "故而", "旁人", "无", "无宁", "无论", "既", "既往", "既是", "既然", "日", "时", "时候", "是", "是以", "是的", "更", "曾", "替", "替代", "最", "月", "有", "有些", "有关", "有及", "有时", "有的", "望", "朝", "朝着", "本", "本人", "本地", "本着", "本身", "来", "来着", "来自", "来说", "极了", "果然", "果真", "某", "某个", "某些", "某某", "根据", "欤", "正值", "正如", "正巧", "正是", "此", "此地", "此处", "此外", "此时", "此次", "此间", "毋宁", "每", "每当", "比", "比及", "比如", "比方", "没奈何", "沿", "沿着", "漫说", "点", "焉", "然则", "然后", "然而", "照", "照着", "犹且", "犹自", "甚且", "甚么", "甚或", "甚而", "甚至", "甚至于", "用", "用来", "由", "由于", "由是", "由此", "由此可见", "的", "的确", "的话", "直到", "相对而言", "省得", "看", "眨眼", "着", "着呢", "矣", "矣乎", "矣哉", "离", "秒", "称", "竟而", "第", "等", "等到", "等等", "简言之", "管", "类如", "紧接着", "纵", "纵令", "纵使", "纵然", "经", "经过", "结果", "给", "继之", "继后", "继而", "综上所述", "罢了", "者", "而", "而且", "而况", "而后", "而外", "而已", "而是", "而言", "能", "能否", "腾", "自", "自个儿", "自从", "自各儿", "自后", "自家", "自己", "自打", "自身", "至", "至于", "至今", "至若", "致", "般的", "若", "若夫", "若是", "若果", "若非", "莫不然", "莫如", "莫若", "虽", "虽则", "虽然", "虽说", "被", "要", "要不", "要不是", "要不然", "要么", "要是", "譬喻", "譬如", "让", "许多", "论", "设使", "设或", "设若", "诚如", "诚然", "该", "说", "说来", "请", "诸", "诸位", "诸如", "谁", "谁人", "谁料", "谁知", "贼死", "赖以", "赶", "起", "起见", "趁", "趁着", "越是", "距", "跟", "较", "较之", "边", "过", "还", "还是", "还有", "还要", "这", "这一来", "这个", "这么", "这么些", "这么样", "这么点儿", "这些", "这会儿", "这儿", "这就是说", "这时", "这样", "这次", "这般", "这边", "这里", "进而", "连", "连同", "逐步", "通过", "遵循", "遵照", "那", "那个", "那么", "那么些", "那么样", "那些", "那会儿", "那儿", "那时", "那样", "那般", "那边", "那里", "都", "鄙人", "鉴于", "针对", "阿", "除", "除了", "除外", "除开", "除此之外", "除非", "随", "随后", "随时", "随着", "难道说", "零", "非", "非但", "非徒", "非特", "非独", "靠", "顺", "顺着", "首先", "︿", "！", "＃", "＄", "％", "＆", "（", "）", "＊", "＋", "，", "０", "１", "２", "３", "４", "５", "６", "７", "８", "９", "：", "；", "＜", "＞", "？", "＠", "［", "］", "｛", "｜", "｝", "～", "￥"]
    )
    stopwords = stopwords.union(set(string.punctuation))
    
    constraints = [
        textattack.constraints.pre_transformation.RepeatModification(), 
        textattack.constraints.pre_transformation.StopwordModification(stopwords=stopwords)
    ]

    input_column_modification = textattack.constraints.pre_transformation.InputColumnModification(
        ["premise", "hypothesis"], {"premise"}
    )
    constraints.append(input_column_modification)
    constraints.append(textattack.constraints.grammaticality.PartOfSpeech(allow_verb_noun_swap=False))

    muse_constraint = textattack.constraints.semantics.sentence_encoders.MultilingualUniversalSentenceEncoder(
        threshold=0.90,
        metric="cosine",
        compare_against_original=False,
        window_size=20,
        skip_text_shorter_than_window=False,
    )
    constraints.append(muse_constraint)

    goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper, query_budget=10000)

    #search_method = textattack.search_methods.BeamSearch(beam_width=4)
    search_method = textattack.search_methods.GreedyWordSwapWIR()

#     word_segmenter = textattack.shared.ChineseWordSegmenter()

    return textattack.Attack(goal_function, constraints, transformations, search_method)

#     return textattack.Attack(goal_function, constraints, transformations, search_method, word_segmenter=word_segmenter)


In [24]:
model = transformers.BertForSequenceClassification.from_pretrained("hfl/chinese-bert-wwm-ext")
tokenizer = transformers.BertTokenizerFast.from_pretrained("hfl/chinese-bert-wwm-ext")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)
attack = ChineseMorphemeAttack(model_wrapper)

print(attack)

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

Important data file "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/OpenHowNet/HowNet_dict_complete" lost, please run `OpenHowNet.download()`.


Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


5 out of the last 5 calls to <function recreate_function.<locals>.restored_function_body at 0x7fcf410dd680> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapWordNet
    (1): ChineseWordSwapHowNet
    (2): WordSwapMaskedLM(
        (method):  bae
        (masked_lm_name):  BertForMaskedLM
        (max_length):  128
        (max_candidates):  20
        (min_confidence):  0.0
      )
    )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  False
        (compare_against_original):  True
      )
    (1): MultilingualUniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.9
        (window_size):  20
        (skip_text_shorter_than_window):  False
        (compare_against_original):  False
      )
    (2): RepeatModification
    (3): StopwordModification
    (4): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (columns_to

In [33]:
import argparse
import datasets
import functools


custom_dataset = [
    ('大家好我的名字是蔡程远', '大家好', 1), #A negative comment, with a negative label
    ('大家好我的名字是蔡程远', '大家好', 1), #A negative comment, with a positive label
    ('大家好我的名字是蔡程远', '大家好', 1), #A positive comment, with a negative label
    ('大家好我的名字是蔡程远', '大家好', 1), #A positive comment, with a positive label
    ('大家好我的名字是蔡程远', '大家好', 1), #A positive comment, with a positive label
]

dataset = textattack.datasets.load_dataset('csv', data_files="./data/zh_sentiment/test.tsv", delimiter="\t")["train"]
dataset = textattack.datasets.HuggingFaceDataset(
            "zh-sentiment",
            lang="zh",
            dataset_columns=(["text"], "label"),
            label_names=["Negative", "Positive"]
        )
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker, AttackArgs

attack_args = AttackArgs(num_successful_examples=5, log_to_csv="results.csv", csv_coloring_style="html")
attacker = Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

AttributeError: module 'textattack.datasets' has no attribute 'load_dataset'